# **2110443 - Computer Vision (2022/2)**
#**Lab 10 - Computer Vision on the Edge** <br>
In this lab, we will learn how to deploy deep learning models on edge platform (Jetson Nano) by using proper method.

# 1. Image classification model deployment (Chest X-Ray Images (Pneumonia))
This dataset is taken from https://www.kaggle.com/paultimothymooney/chest-xray-pneumonia
![Dataset samples](https://i.imgur.com/jZqpV51.png)

The dataset is organized into 3 folders (train, test, val) and contains subfolders for each image category (Pneumonia/Normal). There are 5,863 X-Ray images (JPEG) and 2 categories (Pneumonia/Normal).

Chest X-ray images (anterior-posterior) were selected from retrospective cohorts of pediatric patients of one to five years old from Guangzhou Women and Children’s Medical Center, Guangzhou. All chest X-ray imaging was performed as part of patients’ routine clinical care.

In [ ]:
!pip install timm

In [ ]:
import random
import numpy as np
import cv2
from matplotlib import pyplot as plt
from tqdm.notebook import tqdm

import timm
import torch
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from torchvision import transforms
from torchvision import models as models
from sklearn.metrics import confusion_matrix

# To guarantee reproducible results 
torch.manual_seed(2)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(2)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## 1.1 GPU status check

In [ ]:
!nvidia-smi

## 1.2 Download and inspect pneumonia chest-xray dataset

In [ ]:
!wget -O chest_xray.zip https://www.piclab.ai/classes/cv2020/chest_xray.zip
!unzip -qo chest_xray.zip

In [ ]:
### Helper function to display image from dataset ###
def getImageFromDataset(dataset, idx):
  sampleImage, sampleLabel = dataset.__getitem__(idx)
  ### Revert transformation ###
  sampleImage = ((sampleImage.permute(1,2,0).numpy() * np.array([0.229, 0.224, 0.225])) + np.array([0.485, 0.456, 0.406]))*255
  sampleImage = sampleImage.astype(np.uint8)
  sampleClassName = dataset.classes[sampleLabel]
  return sampleImage, sampleClassName

In [ ]:
### Dataset Augmentation (https://pytorch.org/docs/stable/torchvision/transforms.html) ###
transformTrain = transforms.Compose([        
        transforms.RandomResizedCrop(size=256, scale=(0.8, 1.0)),
        transforms.RandomRotation(degrees=15),
        transforms.ColorJitter(),
        transforms.RandomHorizontalFlip(),
        transforms.CenterCrop(size=(224,224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],[0.229, 0.224, 0.225])])

transformTest =  transforms.Compose([
        transforms.Resize(size=(224,224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

### Dataloader for our dataset ###
pneumoniaTrainDataset = ImageFolder('chest_xray/train/', transform=transformTrain)
pneumoniaTestDataset = ImageFolder('chest_xray/test/', transform=transformTest)

print('Total train set images :', len(pneumoniaTrainDataset))
print('Total test set images :', len(pneumoniaTestDataset))

## 1.3 Dataset visualization

In [ ]:
normalImage, normalClassName = getImageFromDataset(pneumoniaTrainDataset, 0)
pneumoniaImage, pneumoniaClassName = getImageFromDataset(pneumoniaTrainDataset, 3000)


_, figure = plt.subplots(1,2)

figure[0].imshow(normalImage,cmap='gray')
figure[0].title.set_text(normalClassName)

figure[1].imshow(pneumoniaImage,cmap='gray')
figure[1].title.set_text(pneumoniaClassName)
plt.show()

## 1.4 Construct the model from pretrained network (timm), optimizer and loss function

timm documentation : https://rwightman.github.io/pytorch-image-models/


In [ ]:
pneuNet = timm.create_model('mobilenetv3_large_100', pretrained=True, num_classes=2)

In [ ]:
pneuNet.cuda()

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(pneuNet.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=5, verbose=True)

pneumoniaTrainDatasetLoader = DataLoader(pneumoniaTrainDataset, batch_size=16, shuffle=True, num_workers=2, pin_memory=True)
pneumoniaTestDatasetLoader = DataLoader(pneumoniaTestDataset, batch_size=16, shuffle=False, num_workers=2, pin_memory=True)

## 1.5 Train the model

In [ ]:
### Train and test helper function ###
def testModel(testDatasetLoader, net):
  net.eval()
  correctImages = 0
  totalImages = 0
  allLabels = []
  allPredicted = []
  testingProgressbar = tqdm(enumerate(testDatasetLoader), total=len(testDatasetLoader), ncols=100)
  with torch.no_grad():
    for batchIdx, batchData in testingProgressbar:
      images, labels = batchData
      
      images, labels = images.cuda(), labels.cuda()
      outputs = net(images)
      _, predicted = torch.max(outputs, 1)

      correctImages += (predicted == labels).sum().item()
      totalImages += labels.size(0)

      accumulateAccuracy = round((correctImages/totalImages)*100,4)
      testingProgressbar.set_description("Testing accuracy: {}".format(accumulateAccuracy ) )
    
      allLabels.append(labels)
      allPredicted.append(predicted)
  allLabels = torch.cat(allLabels).cpu().numpy()
  allPredicted = torch.cat(allPredicted).cpu().numpy()
  return correctImages, totalImages, allLabels, allPredicted

def trainAndTestModel(trainDatasetLoader, testDatasetLoader, net, optimizer,scheduler, criterion, trainEpoch):
  
  bestAccuracy = 0
  correctImages = 0
  totalImages = 0
  for currentEpoch in tqdm(range(trainEpoch), desc='Overall Training Progress:', ncols=100):
    trainingLoss = 0.0
    net.train()
    print('Epoch',str(currentEpoch+1),'/',str(trainEpoch))
    trainingProgressbar = tqdm(enumerate(trainDatasetLoader), total=len(trainDatasetLoader), ncols=100)
    for batchIdx, batchData in trainingProgressbar:
      images, labels = batchData
      images, labels = images.cuda(), labels.cuda()

      # zero the parameter gradients
      optimizer.zero_grad()

      # forward + backward + optimize
      outputs = net(images)
      loss = criterion(outputs, labels)
    
      _, predicted = torch.max(outputs, 1)
      correctImages += (predicted == labels).sum().item()
      totalImages += labels.size(0)
    
      loss.backward()
      optimizer.step()
      

      trainingLoss += loss.item()
      accumulateAccuracy = round((correctImages/totalImages)*100,4)
      trainingProgressbar.set_description("Training accuracy: {} loss: {}".format(accumulateAccuracy, round(loss.item(),4) ) )
    scheduler.step(trainingLoss)
    correctImages, totalImages, allLabels, allPredicted = testModel(testDatasetLoader, net)
    testAccuracy = round((correctImages/totalImages)*100,2)

    print('='*10)
    
    if testAccuracy > bestAccuracy:
      bestAccuracy = testAccuracy
      bestPredicted = allPredicted
      bestNet = net

  return bestAccuracy, bestPredicted, allLabels, bestNet

In [ ]:
### TODO : Train the model by using trainAndTestModel function ###
bestAccuracy, bestPredicted, allLabels, bestNet = trainAndTestModel(pneumoniaTrainDatasetLoader, pneumoniaTestDatasetLoader, 
                                                                    pneuNet, 
                                                                    optimizer, scheduler, criterion, 
                                                                    trainEpoch=3)

## 1.6 Find the confusion matrix and calculate TP, TN, FP, and FN

In [ ]:
### Confusion matrix plot helper function from https://www.kaggle.com/grfiv4/plot-a-confusion-matrix ###
def plot_confusion_matrix(cm,target_names,title='Confusion matrix',cmap=None,normalize=True):
    import matplotlib.pyplot as plt
    import numpy as np
    import itertools

    accuracy = np.trace(cm) / float(np.sum(cm))
    misclass = 1 - accuracy

    if cmap is None:
        cmap = plt.get_cmap('Blues')

    plt.figure(figsize=(8, 6))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()

    if target_names is not None:
        tick_marks = np.arange(len(target_names))
        plt.xticks(tick_marks, target_names, rotation=45)
        plt.yticks(tick_marks, target_names)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]


    thresh = cm.max() / 1.5 if normalize else cm.max() / 2
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        if normalize:
            plt.text(j, i, "{:0.4f}".format(cm[i, j]),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")
        else:
            plt.text(j, i, "{:,}".format(cm[i, j]),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")
            
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label\naccuracy={:0.4f}; misclass={:0.4f}'.format(accuracy, misclass))
    plt.show()

In [ ]:
confusionMatrix = confusion_matrix(allLabels, bestPredicted)
plot_confusion_matrix(cm           = confusionMatrix, 
                      normalize    = False,
                      target_names = pneumoniaTrainDataset.classes,
                      title        = "Pneumonia Classification Confusion Matrix")
tn, fp, fn, tp = confusionMatrix.ravel()
print('TP:{} TN:{} FP:{} FN:{}'.format(tn, fp, fn, tp))

## 1.7 Save - Load Model in PyTorch and Model usage after training

In [ ]:
# Save
torch.save(pneuNet.state_dict(), '/content/drive/MyDrive/chestxray.pth')

In [ ]:
# Load
weightDict = torch.load('/content/drive/MyDrive/chestxray.pth', map_location='cpu')

In [ ]:
pneuNet = timm.create_model('mobilenetv3_large_100', pretrained=True, num_classes=2)
pneuNet.load_state_dict(weightDict)
pneuNet.cuda()
pneuNet.eval();

In [ ]:
inputImage = cv2.imread('chest_xray/val/PNEUMONIA/person1946_bacteria_4875.jpeg')
inputImage = cv2.resize(cv2.cvtColor(inputImage, cv2.COLOR_BGR2RGB), (224,224))
inputTensor = ((inputImage / 255) - [0.485, 0.456, 0.406]) / [0.229, 0.224, 0.225]

inputTensor = torch.from_numpy(inputTensor.astype(np.float32))
print('Before permute', inputTensor.shape)
inputTensor = inputTensor.permute(2,0,1).unsqueeze(0)
print('After permute and unsqueeze', inputTensor.shape)

In [ ]:
with torch.no_grad():
  inputTensor = inputTensor.cuda()
  output = pneuNet(inputTensor)
  _, predicted = torch.max(output, 1)
  print('Result', output, predicted)

## 1.8 Export to ONNX
ONNX graph can be visualized by using [netron](https://netron.app)

In [ ]:
# Export the model
torch.onnx.export(pneuNet,               # model being run
                  inputTensor,           # model input (or a tuple for multiple inputs)
                  "chestxray.onnx",   # where to save the model (can be a file or file-like object)
                  export_params=True,        # store the trained parameter weights inside the model file
                  opset_version=12,          # the ONNX version to export the model to
                  do_constant_folding=True,  # whether to execute constant folding for optimization
                  input_names = ['input'],   # the model's input names
                  output_names = ['output'], # the model's output names
                  dynamic_axes={'input' : {0 : 'batch_size'},    # variable length axes
                                'output' : {0 : 'batch_size'}})

## 1.9 Run inference using ONNXRuntime

In [ ]:
!pip install onnxruntime

In [ ]:
import onnxruntime as rt

sessOptions = rt.SessionOptions()
sessOptions.graph_optimization_level = rt.GraphOptimizationLevel.ORT_ENABLE_ALL 
chestxrayModel = rt.InferenceSession('chestxray.onnx', sessOptions)

In [ ]:
inputImage = cv2.imread('chest_xray/val/PNEUMONIA/person1946_bacteria_4875.jpeg')
inputImage = cv2.resize(cv2.cvtColor(inputImage, cv2.COLOR_BGR2RGB), (224,224))
inputTensorNp = ((inputImage / 255) - [0.485, 0.456, 0.406]) / [0.229, 0.224, 0.225]

inputTensorNp = inputTensorNp.transpose(2,0,1)[np.newaxis].astype(np.float32)
print('After permute and unsqueeze', inputTensorNp.shape)

In [ ]:
outputRT = chestxrayModel.run([], {'input': inputTensorNp})[0]
print(outputRT)

# 2.Object detection model deployment

In [ ]:
### Restart session (clear GPU ram)
import os
os.kill(os.getpid(), 9)

In this lab, we will learn how to use deploy an object detection model trained by MMDetection.

<br>Install prerequisite libraries for MMDetection
![mmdetection](https://raw.githubusercontent.com/open-mmlab/mmdetection/master/resources/mmdet-logo.png)

In [ ]:
!pip install torch==1.8.0+cu111 torchvision==0.9.0+cu111 torchaudio==0.8.0 -f https://download.pytorch.org/whl/torch_stable.html

In [ ]:
!pip install openmim
!mim install mmdet
!pip install mmpycocotools

## 2.1 Raccoon Dataset
![Raccoon Dataset](https://i.imgur.com/cRQJ1PB.png)

Dataset URL : https://github.com/datitran/raccoon_dataset <br>
This dataset contains 196 images of raccoons and 213 bounding boxes (some images contain two raccoons). This is a single class problem, and images various size and scene condition. It's a great first dataset for getting started with object detection.

Download and extract preprocessed dataset from lab server

In [ ]:
!wget http://piclab.ai/classes/cv2020/raccoonsDataset.zip
!unzip -q raccoonsDataset.zip

### Dataset Exploration
We will use pycocotools to explore this dataset. 

In [ ]:
from pycocotools.coco import COCO
import numpy as np
import cv2
import matplotlib.pyplot as plt

trainLabelFile='raccoons/coco_annotations.json'
# initialize COCO api for instance annotations
trainCOCOBinding = COCO(trainLabelFile)

#display COCO categories and supercategories
cats = trainCOCOBinding.loadCats(trainCOCOBinding.getCatIds())
nms=[cat['name'] for cat in cats]
print('COCO categories: \n{}\n'.format(' '.join(nms)))

# get all images containing given categories, select one at random
catIds = trainCOCOBinding.getCatIds(catNms=['raccoon']);
imgIds = trainCOCOBinding.getImgIds(catIds=catIds );

randomImgId = np.random.randint(0,len(imgIds))
sampleImageData = trainCOCOBinding.loadImgs(imgIds[randomImgId])[0]

print('Image Data >>', sampleImageData)

sampleImage = cv2.imread('raccoons/'+sampleImageData['file_name'])

annIds = trainCOCOBinding.getAnnIds(imgIds=randomImgId, catIds=catIds, iscrowd=None)
boxes = trainCOCOBinding.loadAnns(annIds)
print('Box Data', boxes)

for box in boxes:
  x,y,w,h = box['bbox']
  cv2.rectangle(sampleImage, (int(x), int(y)), (int(x+w), int(y+h)), (0,255,0), 5)

sampleImage = cv2.cvtColor(sampleImage, cv2.COLOR_BGR2RGB)

plt.imshow(sampleImage)
plt.show()

## 2.2 **Modify MMDetection model configuration**

In [ ]:
from mmcv import Config

!mim download mmdet --config retinanet_regnetx-800MF_fpn_1x_coco --dest .

modelConfig = Config.fromfile('retinanet_regnetx-800MF_fpn_1x_coco.py') 

print(f'Original Config:\n{modelConfig.pretty_text}')

In [ ]:
from mmcv.utils.config import ConfigDict
from mmdet.apis import set_random_seed


modelConfig.device = 'cuda'

# Modify dataset type and path
modelConfig.dataset_type = 'CocoDataset'
modelConfig.data_root = './raccoons'
modelConfig.classes = ('raccoon',)

modelConfig.data.train = ConfigDict()
modelConfig.data.train.pipeline = modelConfig.train_pipeline
modelConfig.data.train.type = 'CocoDataset'
modelConfig.data.train.classes = ('raccoon',)
modelConfig.data.train.data_root = './raccoons'
modelConfig.data.train.ann_file = 'coco_annotations.json'
modelConfig.data.train.img_prefix = ''

modelConfig.data.val = ConfigDict()
modelConfig.data.val.pipeline = modelConfig.test_pipeline
modelConfig.data.val.type = 'CocoDataset'
modelConfig.data.val.classes =('raccoon',)
modelConfig.data.val.data_root = './raccoons'
modelConfig.data.val.ann_file = 'coco_annotations.json'
modelConfig.data.val.img_prefix = ''


modelConfig.data.test = ConfigDict()
modelConfig.data.test.pipeline = modelConfig.test_pipeline
modelConfig.data.test.type = 'CocoDataset'
modelConfig.data.test.classes =('raccoon',)
modelConfig.data.test.data_root = './raccoons'
modelConfig.data.test.ann_file = 'coco_annotations.json'
modelConfig.data.test.img_prefix = ''


# Set up working dir to save files and logs.
modelConfig.work_dir = './experiments'
# use pretrained model as start point
modelConfig.load_from = 'retinanet_regnetx-800MF_fpn_1x_coco_20200517_191403-f6f91d10.pth'

modelConfig.optimizer.lr = 1e-3
modelConfig.lr_config.warmup = None
modelConfig.lr_config.policy = 'step'
modelConfig.lr_config.step = [10,25]
modelConfig.log_config.interval = 10


# Modify num classes of the model in box head
modelConfig.model.bbox_head.num_classes = 1
# Evaluation interval
modelConfig.evaluation.interval = 5
# Checkpoint saving interval
modelConfig.checkpoint_config.interval = 5
modelConfig.runner.max_epochs = 10

# Set seed thus the results are more reproducible
modelConfig.seed = 0
set_random_seed(0, deterministic=False)
modelConfig.gpu_ids = range(1)

# We can initialize the logger for training and have a look
# at the final config used for training
print(f'Modified Config:\n{modelConfig.pretty_text}')

##2.3 Training

In [ ]:
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector
import mmcv
import os

# Build dataset
datasets = [build_dataset(modelConfig.data.train)]

# Build the detector
model = build_detector(modelConfig.model, train_cfg=modelConfig.get('train_cfg'), test_cfg=modelConfig.get('test_cfg'))

# Create work_dir
mmcv.mkdir_or_exist(os.path.abspath(modelConfig.work_dir))
train_detector(model, datasets, modelConfig, distributed=False, validate=True)

##2.4 Inference on image!

In [ ]:
from mmdet.apis import inference_detector, init_detector, show_result_pyplot
inputImage = mmcv.imread('raccoons/raccoon-115_jpg.rf.9723b0a68ad8ed8bdb5ccf6a210ba09b.jpg')

model.cfg = modelConfig
model.CLASSES = ('raccoons',)

result = inference_detector(model, inputImage)
show_result_pyplot(model, inputImage, result)

## 2.5 Export to ONNX
List of exportable model can be read from [here](https://mmdetection.readthedocs.io/en/v2.11.0/tutorials/pytorch2onnx.html)

In [ ]:
!pip install onnx onnxruntime
!mim install mmdeploy

In [ ]:
import mmdeploy
print(mmdeploy.__version__)

In [ ]:
!git clone --depth 1 --branch v0.2.0 https://github.com/open-mmlab/mmdeploy/

In [ ]:
!python mmdeploy/tools/deploy.py \
    mmdeploy/configs/mmdet/detection/detection_onnxruntime_dynamic.py \
    config.py \
    experiments/latest.pth \
    raccoons/raccoon-115_jpg.rf.9723b0a68ad8ed8bdb5ccf6a210ba09b.jpg \
    --work-dir output \
    --show

##2.6 Run inference using ONNXRuntime

In [ ]:
import onnxruntime as rt
import cv2
import numpy as np
from matplotlib import pyplot as plt

sessOptions = rt.SessionOptions()
sessOptions.graph_optimization_level = rt.GraphOptimizationLevel.ORT_ENABLE_ALL 
raccoonModel = rt.InferenceSession('/content/output/end2end.onnx', sessOptions)

In [ ]:
inputImage = cv2.imread('raccoons/raccoon-115_jpg.rf.9723b0a68ad8ed8bdb5ccf6a210ba09b.jpg')
ratioH, ratioW = inputImage.shape[0] / 320, inputImage.shape[1] / 320


inputTensor = cv2.resize(inputImage, (320,320))
inputTensor = (inputTensor - [103.53, 116.28, 123.675]) / [57.375, 57.12, 58.395]
inputTensor = inputTensor.transpose(2,0,1)[np.newaxis].astype(np.float32)
print('After permute and unsqueeze', inputTensor.shape)

In [ ]:
outputRT = raccoonModel.run([], {'input': inputTensor})
outputBoxes, outputLabels = outputRT
print(outputBoxes.shape, outputLabels.shape)

In [ ]:
outputImage = cv2.cvtColor(inputImage, cv2.COLOR_BGR2RGB)

rescaleOutputBoxes = outputBoxes * [ratioW, ratioH, ratioW, ratioH, 1]
for boxData in zip(rescaleOutputBoxes[0], outputLabels[0]):

  prob  = boxData[0][4]
  if prob > 0.5:
    x1 = int(boxData[0][0])
    y1 = int(boxData[0][1])
    x2 = int(boxData[0][2])
    y2 = int(boxData[0][3])
    label = boxData[1]
    cv2.rectangle(outputImage, (x1,y1), (x2,y2), (0,255,0), 3)
  
plt.figure(figsize=(10,10))
plt.imshow(outputImage)
plt.show()